In [21]:
from flask import Flask, render_template, request
import re
import sys
import os
import torch

In [22]:
sys.path.append(os.path.abspath('model'))
sys.path.append(os.path.abspath('templates'))


In [23]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import os
import sys

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )

        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(128, 10),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        # flaten tensor
        x = x.view(x.size(0), -1)
        return self.fc(x)


def init():
    model = Net()
    sys.path.append(os.path.abspath('model'))
    path = os.path.join('model', 'model' + "." + 'pt')
    print(os.path.basename(path))
    print(path)
    path = "C:\\Users\\Shudipto\\PycharmProjects\\DeployBNDegit\\model\\model.pt"
    return load_checkpoint(model, path)


def load_checkpoint(model, path):
    # Make sure to set parameters as not trainable
    for param in model.parameters():
        param.requires_grad = False

    print("Path again:", path)
    # Load in checkpoint
    if torch.cuda.is_available():
        checkpoint = torch.load(path)
    else:
        checkpoint = torch.load(path, map_location="cpu")

    # Extract classifier
    classifier = checkpoint['classifier']
    # set classifier
    try:
        check = model.classifier
    except AttributeError:
        check = False

    if check is not False:
        model.classifier = classifier
    else:
        model.fc = classifier

    # Extract others
    model.cat_to_name = checkpoint['class_to_name']
    model.epochs = checkpoint['epochs']

    # Load in the state dict
    model.load_state_dict(checkpoint['state_dict'])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    return model


def transform(file):
    img = Image.open(file).convert('L')

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    test_transform = transforms.Compose([
        transforms.Resize(180),
        transforms.ToTensor(),
        #transforms.Normalize(mean, std)
    ])

    img = test_transform(img).unsqueeze(0)
    return img


In [18]:
os.listdir('C:\\Users\\Shudipto\\PycharmProjects\\DeployBNDegit\\model')

['load.py', 'model.pt', 'test.py', '__pycache__']

In [25]:
import base64

# init flask app
app = Flask(__name__, template_folder='C:\\Users\\Shudipto\\PycharmProjects\\DeployBNDegit\\templates')

global model

model = init()


def convert_image(x):
    x = x.replace('data:image/png;base64,'.encode(), ''.encode())
    with open("output.png", "wb") as fh:
        fh.write(base64.decodebytes(x))


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/predict/', methods=['GET', 'POST'])
def predict():
    imdata = request.get_data()
    print('Get Data:', imdata)
    convert_image(imdata)
    x = transform('output.png')
    with torch.no_grad():
        out = model(x)
        proba = torch.exp(out)
        top_p, top_class = proba.topk(1, dim=1)
        value = top_class.item()
        return str(value)


if __name__ == '__main__':
    app.run()



model.pt

model\model.pt

Path again:

C:\Users\Shudipto\PycharmProjects\DeployBNDegit\model\model.pt

 * Serving Flask app "__main__" (lazy loading)


 * Environment: production


   Use a production WSGI server instead.


 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)